# <center> Бустинг </center>

### План

1. #### Теория
    - пример для иллюстрации принципа работы бустинга
    - библиотеки: Sklearn, XgBoost, LightGBM, CatBoost
2. #### Практика
    - Пример запуска sklearn.ensemble.GradientBoostingClassifier
    - Пример запуска LightGBM
    - Сравнение GradientBoostingClassifier и LightGBM
3. #### Выводы
4. #### Дополнительный материал: соревнование Porto Seguro Safe Driver's Prediction

<img src='img/boosting.jpg' width=300>

# Теория

### Пример для иллюстрации принципа работы бустинга
Пусть базовые алгоритмы – всего лишь "пеньки", то есть деревья решений глубины 1.
<img src='img/adaboost_toy_step1.png' width=700>
Веса объектов, на которых базовый алгоритм ошибается, увеличиваются (кружки увеличиваются в размере).
<img src='img/adaboost_toy_step2.png' width=700>
В конце базовые алгоритмы "голосуют", их веса определялись $\alpha_t$ в процессе построения.
<img src='img/adaboost_toy_step3.png' width=700>

### Библиотеки

<img src='img/timeline.png'>

#### XgBoost
- есть JVM, Julia, R package
- не умеет обрабатывать категориальные признаки (в отличие от LightGBM и CatBoost)
- зачастую ставится на Windows с большими проблемами

#### LightGBM
- умеет обрабатывать категориальные признаки
- эффективно расходует память
- достаточно часто дает качество чуть лучше, чем CatBoost
 
#### CatBoost
- можно добавить регуляризацию (помогает избежать переобучения)
- есть C и C++ API
- можно использовать красивые средства визуализации: CatBoost Viewer и TensorBoard
- можно прикрутить к ClickHouse
- feature/object importance (можно считать важность не только признаков, но и объектов в выборке)
- делает бэкапы в процессе обучения (если что-то пойдет не так, не нужно будет переучивать модель заново)

# Практика

В sklearn доступны алгоритмы AdaBoost и GradientBoosting для задач классификации и регрессии.
Рассмотрим пример запуска <a href="http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html">GradientBoostingClassifier</a>.

In [122]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
import lightgbm

import warnings
warnings.simplefilter("ignore")

## 1. Пример запуска sklearn.ensemble.GradientBoostingClassifier
### <a href='http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html'> Список параметров </a>
- #### метрики
    - loss [default="deviance"] - оптимизируемая функция потерь.  Одна из {"deviance", "exponential"}. Первая соответствует логистической регрессии и возвращает вероятности, вторая - AdaBoost.
- #### процесс обучения
    - learning_rate [default=0.1] - темп обучения, насколько быстро будут меняться веса.
    - n_estimators [default=100] - число итераций градиентного бустинга.
- #### характеристики деревьев в композиции
    - max_depth [default=3] - максимальная глубина деревьев в композиции.
    - min_samples_split [default=2] - минимальное число примеров, необходимое для разветвления в данной вершине.
    - min_samples_leaf [default=1] - минимальное число примеров в листе.
    - min_weight_fraction_leaf [default=0.0] - минимальное взвешенное число примеров в листе.
- #### технические параметры
    - verbose [default=0] - нужно ли печатать отладочный вывод или нет.

### Генерирация датасета для обучения

Будем генерировать выборку из 1000 объектов и 50 признаков с помощью `sklearn.datasets.make_classification`

In [93]:
# Generating dataset
X, y = make_classification(n_samples=1000, n_features=50, 
                           n_informative=20)

# splitting dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Обучение GradientBoostingClassifier

In [94]:
# GradientBoostingClassifier initialization
clf = GradientBoostingClassifier(n_estimators=100, max_depth=5)

# GradientBoostingClassifier fitting
clf.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

### Предсказание меток классов

In [95]:
# predict
y_pred = clf.predict(X_test)

# measure accuracy
accuracy = accuracy_score(y_pred, y_test)    

# print accuracy
print(u'Accuracy:', accuracy)

Accuracy: 0.9


## 2. Пример запуска LightGBM

### <a href='https://lightgbm.readthedocs.io/en/latest/Parameters.html'> Список параметров </a>
- #### метрики
    - objective [default="regression"] - оптимизируемая функция потерь. Одна из:
        - regression
        - regression_l1
        - huber
        - fair
        - poisson
        - quantile
        - mape
        - binary
        - multiclass
        - xentropy
        - lambdarank
        - etc.
- #### процесс обучения
    - learning_rate [default=0.1] - темп обучения, насколько быстро будут меняться веса.
    - num_iteration [default=100] - число итераций градиентного бустинга.
- #### характеристики деревьев в композиции
    - max_depth [default=-1 (нет ограничения на глубину)] - максимальная глубина деревьев в композиции.
    - min_gain_to_split [default=0.0] - минимальное значение критерия информативности, необходимое для разветвления в данной вершине.
    - min_data_in_leaf [default=20] - минимальное число примеров в листе.
- #### технические параметры
    - verbosity [default=1] - нужно ли печатать отладочный вывод или нет.

### Генерирация датасета для обучения (аналогично предыдущему примеру)

In [96]:
# Generating dataset
X, y = make_classification(n_samples=1000, n_features=50, n_informative=20)

# splitting dataset into train and test
X_train, X_test, y_train, y_test = # your code here

### Обучение LightGBM

Перед обучением нужно превратить данные в формат, доступный в LightGBM. Делаем это с помощью команды `lightgbm.Dataset`

In [98]:
# Create the LightGBM data containers
train_data = lightgbm.Dataset(
    X_train,
    label=y_train
)

test_data = # your code here

Обучаем модель:

In [102]:
model = lightgbm.train(
    {'num_iterations': 100, 'max_depth': 5},
    train_data,
    valid_sets=test_data,
    num_boost_round=100
)

### Предсказание меток классов

In [105]:
# predict
y_pred = # your code here

In [107]:
y_pred[:5]

array([ 0.95728359,  0.70460998,  0.18385195, -0.00568949,  0.89636035])

Посмотрим на `y_pred`. В отличие от `GradientBoostingClassifier` из `sklearn`, `LightGBM` предсказывает не класс объекта, а его вероятность принадлежности классу. Перед тем, как считать `accuracy`, посчитаем метки классов по вероятностям (считаем, что если вероятность больше $0.5$, то объект принадлежит классу $1$, иначе классу $0$).

In [110]:
# transform probabilities to labels
def binarize(prob):
    return int(prob >= 0.5)

y_pred = list(map(binarize, y_pred))

In [112]:
# measure accuracy
accuracy = # your code here  

# print accuracy
print(u'Accuracy:', accuracy)

Accuracy: 0.805


## Выводы

Основные преимущества LightGBM по сравнению с sklearn.ensembles.GradientBoostingClassifier:
- Помимо деревьев возможно использование линейных моделей в качестве базовых классификаторов.
- Скорость работы (cм. `boosting_solved_with_extra_materials.ipynb`)
- Возможность распараллеливания.
- Значительно больший выбор стандартных функций потерь, а также возможность задавать свою функцию потерь.
- Наличие регуляризаторов в итоговой функции потерь и возможность задавать их коэффициенты, что даёт еще один метод борьбы с переобучением, помимо использования случайности (subsample, colsample_bytree) и основных параметров дерева решений.
- Встроенная обработка missing values.

### Полезные ссылки:

1. <a href='https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db'> Сравнение XgBoost, LightGBM и CatBoost </a>
2. <a href='http://xgboost.readthedocs.io'> Документация XgBoost </a>
3. <a href='https://lightgbm.readthedocs.io/en/latest/'> Документация LightGBM </a>
4. <a href='https://tech.yandex.com/catboost/doc/dg/concepts/about-docpage/'> Документация CatBoost </a>
5. <a href='http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html'> Документация sklearn.ensemble.GradientBoostingClassifier </a>
6. <a href='https://habr.com/company/ods/blog/327250/'> Статья ODS про бустинг </a>
7. <a href='https://alexanderdyakonov.wordpress.com/2017/06/09/градиентный-бустинг/comment-page-1/'> Статья А.Дьяконова про градиентный бустинг </a>
